# Chest Xray Abnormality Classification using Xception-Net and CBAM

Loading required modules

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,2"
# from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(1234)

from tensorflow.keras.layers import Conv3DTranspose as Deconvolution3D


from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv3D, ZeroPadding3D, UpSampling3D, Dense, concatenate, Conv3DTranspose, Cropping3D, PReLU
from tensorflow.keras.layers import MaxPooling3D, GlobalAveragePooling3D, AvgPool3D
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Input
from keras.models import load_model


import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard

from sys import getsizeof
from sklearn.utils import class_weight

IMSIZE = 320,320
RANDOM_SEED = 1234

from numpy.random import seed
seed(1234)
import tensorflow
tensorflow.random.set_seed(1234)

import os
import numpy as np
import cv2
import tensorflow as tf
import pickle
import glob as glob
from sklearn.utils import shuffle
from scipy import ndimage
from tqdm import tqdm
from sklearn import preprocessing

import json
import SimpleITK as sitk
from glob import glob 
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat

import glob
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure, feature
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import imutils
from PIL import Image
import pickle
import pandas as pd
from skimage import data, img_as_float
from skimage import exposure

## Data Loading and Preprocessing

1- Normalization
2- Resizing the image
3- Histogram Equalization to improve the contrast of chest images

In [ ]:
def get_norm(scan):
    m=np.mean(scan)
    st=np.std(scan)
    scan=(scan-m)/st
#     scan = scan.astype(np.float64)
    return scan

def res_scan(nscan):
#     W,H  =  shape[0],shape[1]
#     desired_depth = D
    desired_width = 320
    desired_height = 320
#     current_depth = nscan.shape[0]
    current_width = nscan.shape[0]
    current_height = nscan.shape[1]
#     depth_factor = desired_depth/current_depth
    width_factor = desired_width/current_width
    height_factor = desired_height/current_height
    nscan = ndimage.zoom(nscan,  ( width_factor,height_factor), order=1)
    return nscan

def preprocess_image(sc):
    n = img_as_float(sc)
    n = exposure.equalize_adapthist(n/np.max(n))
    n=res_scan(n)
    n=get_norm(n)
    return n

In [ ]:
def load_samples(csv_file):
    data = pd.read_csv(csv_file)
    data=shuffle(data)
    data = data[['Path', 'No Finding']]
    file_names = list(data.iloc[:,0])
    # Get the labels present in the second column
    labels = list(data.iloc[:,1])
    samples=[]
    for samp,lab in zip(file_names,labels):
        samples.append([samp,lab])
    return samples

In [ ]:
from tensorflow.keras.utils import Sequence

## Datagenerator

Data generator allows to load one batch of images at a time thus it omits the need of loading all the images at the same time and do not occupy large RAM.

In [ ]:
class DataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, list_IDs,to_fit=True, batch_size=32, dim=(8 ,512, 512), shuffle=True):
        """Initialization
        :param list_IDs: list of all 'label' ids to use in the generator
        :param labels: list of image labels (file names)
        :param image_path: path to images location
        :param mask_path: path to masks location
        :param to_fit: True to return X and y, False to return X only
        :param batch_size: batch size at each iteration
        :param dim: tuple indicating image dimension
        :param n_channels: number of image channels
        :param n_classes: number of output masks
        :param shuffle: True to shuffle label indexes after every epoch
        """
        self.list_IDs = list_IDs
        self.to_fit = to_fit
        self.batch_size = batch_size
        self.dim = dim
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data

        if self.to_fit:
            X,y = self._generate_Xy(list_IDs_temp)
            return X, y
        else:
            X = self._generate_X(list_IDs_temp)
            return X

    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

            
    def _generate_Xy(self, list_IDs_temp):
        """Generates data containing batch_size images
        :param list_IDs_temp: list of label ids to load
        :return: batch of images
        """
#         global filt

            # Initialise X_train and y_train arrays for this batch
        X_train = []
        y_train = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            img_name = ID[0]
            label = int(ID[1])
            img =  Image.open(img_name)

            # apply any kind of preprocessing
            img = img_as_float(img)
            img = exposure.equalize_adapthist(img/np.max(img))
            img=res_scan(img)
            img=get_norm(img)
            # Add example to arrays
            X_train.append(img)
            y_train.append(label)
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train = np.asarray(y_train).reshape((-1,1))
#         y_test = np.asarray(test_labels).astype('float32').reshape((-1,1))
        return X_train,y_train
            

        

    def _generate_y(self, list_IDs_temp):
        """Generates data containing batch_size masks
        :param list_IDs_temp: list of label ids to load
        :return: batch if masks
        """
        # Initialization
        y = np.empty((self.batch_size, *self.dim,1))
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            y[i,] = self._load_clip_y(ID)
        return y
    
    def _load_clip_y(self, videopath):
        """Load grayscale image
        :param image_path: path to image to load
        :return: loaded image
        """
        global output1_actual
        temp = videopath.split('--')
        frm = int(temp[2])
        clip = np.array([output1_actual[frm:frm+8,:,:,:]])#frames[frm:frm+5,:,:,:]
        return clip
    

    def _load_clip(self, videopath):
        """Load grayscale image
        :param image_path: path to image to load
        :return: loaded image
        """
        global inputs
        temp = videopath.split('--')
        frm = int(temp[2])
        clip = np.array([inputs[frm:frm+8,:,:,:]])#frames[frm:frm+5,:,:,:]
        clip = clip + abs(np.min(clip))
        clip = clip/(np.max(clip))
        return clip

In [ ]:
train_samples = load_samples("/home/amal/Chest Classification/index/train_frt.csv")
validation_samples = load_samples("/home/amal/Chest Classification/index/valid_frt.csv")

In [ ]:
data = pd.read_csv("/home/amal/Chest Classification/index/valid_frt.csv")
data.head()

In [ ]:
train_datagen = DataGenerator(list_IDs=train_samples,to_fit=True, batch_size=12, dim=(320, 320), shuffle=True)
Valid_datagen = DataGenerator(list_IDs=validation_samples,to_fit=True, batch_size=12, dim=(320, 320), shuffle=True)

## Model Defining and Compiling

The Xception-Net is chosen for this problem along with CBAM module.

In [ ]:
import argparse
import time
import random
import cv2
import numpy as np
import keras
import glob

from keras.utils import np_utils
#from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.layers import Conv2D, MaxPooling2D, Flatten, ZeroPadding2D, GlobalAveragePooling2D, concatenate
from keras.layers import BatchNormalization, ReLU, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import tensorflow.keras.backend as K
import tensorflow as tf
from keras.models import Model

from sklearn.utils import class_weight

IMSIZE = 320,320
RANDOM_SEED = 1234

from numpy.random import seed
seed(1234)
import tensorflow
tensorflow.random.set_seed(1234)


In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D
base_model = Xception(weights=None, include_top=False)


input_tensor = Input(shape=(320, 320, 1))
ip = Conv2D(3,(3,3),padding='same')(input_tensor)  
x = base_model(ip)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=input_tensor, outputs=predictions)

In [ ]:
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from keras import backend as K
from keras.activations import sigmoid

def attach_attention_module(net, attention_module):
    if attention_module == 'cbam_block': # CBAM_block
        net = cbam_block(net)
    else:
        raise Exception("'{}' is not supported attention module!".format(attention_module))

    return net

def cbam_block(cbam_feature, ratio=8):
	"""Contains the implementation of Convolutional Block Attention Module(CBAM) block.
	As described in https://arxiv.org/abs/1807.06521.
	"""
	
	cbam_feature = channel_attention(cbam_feature, ratio)
	cbam_feature = spatial_attention(cbam_feature)
	return cbam_feature

def channel_attention(input_feature, ratio=8):
	
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature.shape[channel_axis]
	
	shared_layer_one = Dense(channel//ratio,
							 activation='relu',
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	shared_layer_two = Dense(channel,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	
	avg_pool = GlobalAveragePooling2D()(input_feature)    
	avg_pool = Reshape((1,1,channel))(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	avg_pool = shared_layer_one(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel//ratio)
	avg_pool = shared_layer_two(avg_pool)
	assert avg_pool.shape[1:] == (1,1,channel)
	
	max_pool = GlobalMaxPooling2D()(input_feature)
	max_pool = Reshape((1,1,channel))(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	max_pool = shared_layer_one(max_pool)
	assert max_pool.shape[1:] == (1,1,channel//ratio)
	max_pool = shared_layer_two(max_pool)
	assert max_pool.shape[1:] == (1,1,channel)
	
	cbam_feature = Add()([avg_pool,max_pool])
	cbam_feature = Activation('sigmoid')(cbam_feature)
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
	
	return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
	kernel_size = 7
	
	if K.image_data_format() == "channels_first":
		channel = input_feature.shape[1]
		cbam_feature = Permute((2,3,1))(input_feature)
	else:
		channel = input_feature.shape[-1]
		cbam_feature = input_feature
	
	avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
	assert avg_pool.shape[-1] == 1
	max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
	assert max_pool.shape[-1] == 1
	concat = Concatenate(axis=3)([avg_pool, max_pool])
	assert concat.shape[-1] == 2
	cbam_feature = Conv2D(filters = 1,
					kernel_size=kernel_size,
					strides=1,
					padding='same',
					activation='sigmoid',
					kernel_initializer='he_normal',
					use_bias=False)(concat)	
	assert cbam_feature.shape[-1] == 1
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
		
	return multiply([input_feature, cbam_feature])

In [ ]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
#from attention_module import attach_attention_module

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=4, attention_module=None):
    """ResNet Version 1 Model builder [a]
    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            # attention_module
            if attention_module is not None:
                y = attach_attention_module(y, attention_module)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
attention_module = 'cbam_block'
depth = 50 # For ResNet, specify the depth (e.g. ResNet50: depth=50)
input_shape=320,320,1
model = resnet_v1(input_shape=input_shape, depth=depth, attention_module=attention_module)
adam = optimizers.Adam(lr=0.001, decay=1e-5)
model.compile(optimizer=adam, loss="sparse_categorical_crossentropy",metrics=["sparse_categorical_accuracy"])
log_dir = '/home/azka/chestclassification/'
checkpoint_filepath="/home/azka/CBAM_frt.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
# class_weights={0:1,1:2,2:3,3:5}
batch_size=12
stopping=tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy',
                                          mode='max',
                                          patience=30)
train_steps = (len(train_samples)//batch_size)
valid_steps = (len(validation_samples)//batch_size)
history=model.fit(train_datagen,validation_data=Valid_datagen,epochs=50,batch_size=12,
                  steps_per_epoch=train_steps,
                  validation_steps=valid_steps,
                  callbacks=[model_checkpoint_callback])#,class_weight=class_weights)


## Model Evaluation

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
history.history['val_accuracy']

In [ ]:
model.save_weights("CBAM_actual_weights.hdf5")